[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/style-aligned-colab/blob/main/style_aligned_gradio_colab.ipynb)

In [ ]:
!pip install -q diffusers transformers accelerate peft gradio==3.50.2 mediapy einops

%cd /content
!git clone -b dev https://github.com/camenduru/style-aligned
%cd /content/style-aligned

from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
import mediapy
import sa_handler

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipeline = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, 
                                                     variant="fp16", use_safetensors=False, scheduler=scheduler).to("cuda")

handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(share_group_norm=False, share_layer_norm=False, share_attention=True, 
                                      adain_queries=True, adain_keys=True, adain_values=False)

handler.register(sa_args)

import gradio as gr

def generate(prompt, negative_prompt, num_inference_steps, guidance_scale, width, height):
    image = pipeline(prompt=prompt, negative_prompt=negative_prompt, width=width, height=height, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    return image

with gr.Blocks(title=f"Realtime SDXL Turbo", css=".gradio-container {max-width: 1056px !important}") as demo:
    with gr.Box(scale=23):
      with gr.Row():
        with gr.Column():
            with gr.Row():
              prompt = gr.Textbox(show_label=False, value="a toy train. macro photo. 3d game asset")
              negative_prompt = gr.Textbox(show_label=False, value="low-resolution")
        with gr.Column():
            with gr.Row():
              width = gr.Slider(minimum=256, maximum=2048, step=8, value=1024, label="width")
              height = gr.Slider(minimum=256, maximum=2048, step=8, value=1024, label="height")
        with gr.Column():
            with gr.Row():
              num_inference_steps = gr.Slider(minimum=1, maximum=150, step=1, value=50, label="num_inference_steps")
              guidance_scale = gr.Slider(minimum=2, maximum=25, step=0.5, value=7.5, label="guidance_scale")
      with gr.Column():
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")
          button = gr.Button()
    with gr.Row(variant="default"):
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=1024,
            width=1024,
            elem_id="output_image",
        )

    button.click(fn=generate, inputs=[prompt, negative_prompt, num_inference_steps, guidance_scale, width, height], outputs=[output_image], show_progress=True)

demo.queue().launch(inline=False, share=True, debug=True)